<a href="https://colab.research.google.com/github/kotaro0918/Gemini_RAG/blob/main/Gemini_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

このプログラムを実行する前に、GeminiのAPIキーを発行してシークレットキーに保管してください
またNDCのドキュメントをドライブに保存してください

NDCのtxtデータはEmbeddingのレポジトリに落ちてます
(余裕があったらRAGのレポジトリにも)

In [ ]:
#必要なライブラリのインストール
!pip install langchain
!pip install -q -U google-generativeai

In [ ]:
#Google Driveへのアクセス許可
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#必要なライブラリインポート
# Embedding用
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
# テキストファイルを読み込む
from langchain.document_loaders import TextLoader

In [ ]:
# 環境変数の準備
import google.generativeai as genai
GOOGLE_API_KEY=userdata.get('GEMINI_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
txt_path = "your_txt_path"

In [ ]:
#テキストデータの読み込み
loader = TextLoader(txt_path)
documents = loader.load()

In [ ]:
#テキストデータの分割
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

In [ ]:
df=pd.DataFrame(data=docs,columns=["doc","source","type"])
display(df)

In [ ]:
#分割したドキュメントをEmbedding
docs=list(docs)
result = genai.embed_content(
    model="models/embedding-001",
    content=df["doc"],
    task_type="retrieval_document",
    title="Embedding of list of strings")
df["Embedding"]=result["embedding"]

In [ ]:
#得られたベクトルを標準化
import numpy as np
def normalize_vector(v):
    norm = np.linalg.norm(v)
    if norm == 0:
       return v
    return v / norm

df["Embedding"]=df["Embedding"].map(normalize_vector)
passage_embeddings=df["Embedding"]

In [ ]:
#queryをEmbedding
result = genai.embed_content(
    model="models/embedding-001",
    content="""
    手宮線は、全国で３番目、北海道では最初に開通した路線です。
    南小樽～手宮間は、わずか２．８ｋｍの短い路線ですが、北海道開拓の物資輸送の拠点として発展しました。
    しかし戦後、利用客は減少を続け、昭和６０年１１月、手宮線は廃止されました。
    北海道鉄道発祥の地である手宮駅の跡地は、現在、小樽市総合博物館として活用され現存する日本最古の機関庫とともに、貴重な車両が保存されています。
    """,
    task_type="retrieval_document",
    title="Embedding of list of strings")
query_embedding = normalize_vector(result["embedding"])

In [ ]:
#得られたベクトルの内積を計算
scores = []
for content in passage_embeddings:
  scores.append(np.dot(content,query_embedding))

In [ ]:
#内積を降順に並べ替えた時のインデックスを取得
def get_sorted_indices(lst):
    return [i for i, v in sorted(enumerate(lst), key=lambda x: x[1],reverse = True)]
scores_indices = get_sorted_indices(scores)

In [ ]:
#コサイン類似度の高い３つの文章を出力
print(df["doc"][scores_indices[0:3]])